# Algorytm kryptograficzny RC4 zwany również ARC4
Szyfr strumieniowy to szyfr z kluczem symetrycznym, w którym cyfry w tekście jawnym są łączone ze strumieniem cyfr szyfru pseudolosowego (strumień klucza). W szyfrze strumieniowym każda cyfra tekstu jawnego jest szyfrowana pojedynczo z odpowiadającą jej cyfrą strumienia klucza, aby otrzymać cyfrę strumienia tekstu zaszyfrowanego. Ponieważ szyfrowanie każdej cyfry zależy od bieżącego stanu szyfru, jest ono również znane jako szyfr stanowy. W praktyce cyfrą jest zazwyczaj bit, a operacja łączenia to operacja wykluczająca-lub (XOR).


![RC4](./RC4.png)

Pseudolosowy strumień klucza jest zwykle generowany szeregowo na podstawie losowej wartości początkowej przy użyciu cyfrowych rejestrów przesuwnych. Wartość początkowa służy jako klucz kryptograficzny do deszyfrowania strumienia tekstu zaszyfrowanego. Wykorzystujemy również vektor inicjalizacyjny.

In [16]:
# Implementacja generacji ciągu klucza szyfru RC4

def key_stream_generation(IV, text_lenght, key = b'Vulnerable Secret Key'):
    key = IV + key
    S = [i for i in range(256)]

    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i], S[j] = S[j], S[i]

    i = 0
    j = 0
    keystream = []

    for i in range(text_lenght):
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        keystream.append(K)

    return keystream

Pełna implementacja szyfrowania RC4 w oparciu o ciąg klucza generowanego.
Aby uzyskać szyfrogram wykonujemy operację XOR na kluczach oraz tekście jawnym.

In [17]:
# Implementacja algorytmu RC4

def encrypt(IV, key, plaintext):
  keystream = key_stream_generation(IV, len(plaintext), key)

  ciphertext = [x ^ y for x, y in zip(plaintext, keystream)]
  return (''.join([hex(c)[2:].zfill(2) for c in ciphertext]))

Wykorzystanie szyfru RC4 dla danego klucza, wektora oraz tekstu jawnego.

In [18]:
# Użycie szyfru RC4

key = b'Different Secret Key'
IV = b'Vektor Ini'
plaintext = b'Kryptoanaliza'
cipher = encrypt(IV, key, plaintext)


print("Used key: ", key)
print("USed initialization vector: ", IV)
print("Used plaintext: ",plaintext)
print("Encrypted plaintext: ", cipher)


Used key:  b'Different Secret Key'
USed initialization vector:  b'Vektor Ini'
Used plaintext:  b'Kryptoanaliza'
Encrypted plaintext:  285444737672f33a635d611232


Często klucz wprowadzany do KSA (Key-scheduling Algorithm) składa się ze znanego IV i tajnego klucza. Jeśli zostaną one połączone, wówczas atak Fluhrera, Mantina i Shamira może odzyskać oryginalny klucz poprzez słabe IV (biorąc pod uwagę, że znamy pierwszy bajt wszystkich strumieni kluczy).

![RC4 keystream](./Keystream.png)

Najprostsze słabe IV składają się z 3 bajtów w postaci **(A+3,N−1,X)**, gdzie znamy pierwsze **A** słowo tajnego klucza, **N** wynosi 256 i **X** jest dowolną liczbą.

Aby znaleźć kolejny bajt tajnego klucza, najpierw uruchamiamy A+3 iteracje algorytmu planowania klucza w celu permutacji tablicy S. Zauważ, że pierwsze wartości całego klucza A+3 są nam znane. (IV połączonego z tajnym kluczem).

i oraz j wskaźniki przesuwają się do przodu, zamieniając elementy w taki sposób, abyśmy mogli odzyskać kolejny bajt klucza poprzez równanie:

**Key[i]≡(O−j−S[i])mod256**

**O** jest pierwszym bajtem strumienia klucza, natomiast i oraz j są wskaźnikami z iteracji KSA. Powyższe równanie jest prawdziwe z 5% prawdopodobieństwem, które można łatwo odróżnić od losowego bajtu. Posiadając 60 różnych wartości **X**, mamy ponad 50% szans na odzyskanie kolejnego bajtu klucza.

Podsumowując, możemy odzyskać tajny klucz z dużym prawdopodobieństwem, jeśli zostanie on bezpośrednio dołączony do znanego słabego IV i znamy pierwsze bajty strumienia klucza.

In [19]:
# Implementacja ataku Fluhrer, Mantin, Shamir

def find_key():
  A = 0
  curKey = b''
  results = []

  while A < 253:
      results = []
      for x in range(256):
          # Tworzymy słabe IV
          IV = bytes([A + 3, 255, x])

          # Generacja ciągu klucza przy znanym IV
          # Ta część w przypadku realnego ataku zostałaby wcześniej wygenerowana
          # przez nieświadomego użytkownika
          keystream = key_stream_generation(IV, 64)
          knownKey = IV + curKey

          # Wykorzystanie KSA dla znanych bitów klucza
          S = [i for i in range(256)]
          j = 0
          for i in range(A + 3):
              j = (j + S[i] + knownKey[i % len(knownKey)]) % 256
              S[i], S[j] = S[j], S[i]
          i += 1
          # Zapisanie najprawdopodobnego bitu klucza
          results.append((keystream[0] - j - S[i]) % 256)

      # następny bit klucza zwykle powinnien być tym najczęściej występującym
      nextByte = max(set(results), key = results.count)
      curKey += bytes([nextByte])
      print(f'Current Key: {curKey}')

      # Powtórzenie operacji dla następnego bitu
      A += 1

Wykorzystanie ataku Fluhrer, Mantin, Shamir.

Jak możemy zauważyć atak nie jest w 100% dokładny, jednakże w pierwszych bitach możemy zaobserwować ustalony przez nas wcześniej klucz prywany.


In [20]:
# Użycie ataku Fluhrer, Mantin, Shamir

print(find_key())

Current Key: b'V'
Current Key: b'Vu'
Current Key: b'Vul'
Current Key: b'Vuln'
Current Key: b'Vulne'
Current Key: b'Vulner'
Current Key: b'Vulnera'
Current Key: b'Vulnerab'
Current Key: b'Vulnerabl'
Current Key: b'Vulnerable'
Current Key: b'Vulnerable '
Current Key: b'Vulnerable S'
Current Key: b'Vulnerable Se'
Current Key: b'Vulnerable Sec'
Current Key: b'Vulnerable Secr'
Current Key: b'Vulnerable Secre'
Current Key: b'Vulnerable Secret'
Current Key: b'Vulnerable Secret '
Current Key: b'Vulnerable Secret K'
Current Key: b'Vulnerable Secret Ke'
Current Key: b'Vulnerable Secret Key'
Current Key: b'Vulnerable Secret Key\x18'
Current Key: b'Vulnerable Secret Key\x18s'
Current Key: b'Vulnerable Secret Key\x18s\xc0'
Current Key: b'Vulnerable Secret Key\x18s\xc0L'
Current Key: b'Vulnerable Secret Key\x18s\xc0L\xcb'
Current Key: b'Vulnerable Secret Key\x18s\xc0L\xcb\xf4'
Current Key: b'Vulnerable Secret Key\x18s\xc0L\xcb\xf4\xf0'
Current Key: b'Vulnerable Secret Key\x18s\xc0L\xcb\xf4\xf0C'
Cur

Jakub Jastrzębski, Nicolas Bełz